In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.utils import to_categorical
import librosa
from google.colab import drive
import os

# Mount Google Drive for access
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


Mounted at /content/drive


In [2]:


# Load data from JSON files training with MFCC and overlap
with open('/content/drive/MyDrive/My-207/X_train.json', 'r') as file:
    X_train_list = json.load(file)
    X_train = np.array(X_train_list)

with open('/content/drive/MyDrive/My-207/y_train.json', 'r') as file:
    y_train_list = json.load(file)
    y_train = np.array(y_train_list)

# Load data from JSON files
with open('/content/drive/MyDrive/My-207/X_val.json', 'r') as file:
    X_val_list = json.load(file)
    X_val = np.array(X_val_list)

with open('/content/drive/MyDrive/My-207/y_val.json', 'r') as file:
    y_val_list = json.load(file)
    y_val = np.array(y_val_list)


In [3]:

# Data normalization
scaler = StandardScaler()

# Flatten X_train into a 2-dimensional array
X_train = X_train.reshape(X_train.shape[0], -1)

# Fit and transform the training data
X_train = scaler.fit_transform(X_train)

# Flatten X_val into a 2-dimensional array
X_val = X_val.reshape(X_val.shape[0], -1)

# Transform the validation data using the scaler parameters from the training data
X_val = scaler.transform(X_val)

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# One-hot encode the labels for RNN
y_train_onehot = to_categorical(y_train_encoded)
y_val_onehot = to_categorical(y_val_encoded)

# Save the label encoder for later use
label_encoder_path = "label_encoder.pkl"
import joblib
joblib.dump(label_encoder, label_encoder_path)

# Reshape the data for RNN (add additional dimension for time steps)
X_train_rnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_rnn = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

In [4]:


# Build RNN model topology
model = tf.keras.Sequential([
    SimpleRNN(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True),
    SimpleRNN(64),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])




In [5]:
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3140, 128)         16640     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 29,187
Trainable params: 29,187
Non-trainable params: 0
_________________________________________________________________


In [7]:

# Train the model
history = model.fit(X_train_rnn, y_train_onehot, validation_data=(X_val_rnn, y_val_onehot), batch_size=32, epochs=5)


Epoch 1/5
247/247 [==============================] - 902s 4s/step - loss: 1.1252 - accuracy: 0.3474 - val_loss: 1.1431 - val_accuracy: 0.3393
Epoch 2/5
247/247 [==============================] - 871s 4s/step - loss: 1.1172 - accuracy: 0.3320 - val_loss: 1.1026 - val_accuracy: 0.3412
Epoch 3/5
247/247 [==============================] - 867s 4s/step - loss: 1.1110 - accuracy: 0.3543 - val_loss: 1.1397 - val_accuracy: 0.2842
Epoch 4/5
247/247 [==============================] - 904s 4s/step - loss: 1.1068 - accuracy: 0.3597 - val_loss: 1.1076 - val_accuracy: 0.3098
Epoch 5/5
247/247 [==============================] - 906s 4s/step - loss: 1.0856 - accuracy: 0.3947 - val_loss: 1.1573 - val_accuracy: 0.3201


In [ ]:

# Plot the training and validation loss
plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()
